In [1]:
import thefuzz.fuzz
from thefuzz import process
import re

Простая проверка равенства двух строк

In [2]:
str_A = 'Привет Мир'
str_B = 'Привет миру'
print(str_A == str_B)

False


In [3]:
str_input = input() # ввод строки пользователем
if str_A == str_input:
    print("Строки одинаковые")
else :
    print("Строки разные")

print("Строки одинаковые" if str_A == str_input else "Строки разные") # тернарный оператор if else
print("Строка " + str_A + (" равна " if str_A == str_input else " не равна ") + str_input )
print(f"Строка \"{str_A}\" %s \"{str_input}\""%(" равна " if str_A == str_input else " не равна "))

Строки разные
Строки разные
Строка Привет Мир не равна Привет МИР
Строка "Привет Мир"  не равна  "Привет МИР"


Почему одинаковые по буквам строки могут быть не равны?

In [4]:
print(f"Строка \"{str_A.lower()}\"  %s \"{str_input.lower()}\""%(" равна " if str_A.lower() == str_input.lower() else " не равна ")) # нижний регистр
print(f"Строка \"{str_A.upper()}\" %s \"{str_input.upper()}\""%(" равна " if str_A.lower() == str_input.lower() else " не равна ")) # верхний регистр
print(f"Строка \"{str_A}\" %s \"{str_input}\""%(" равна " if str_A.lower() == str_input.lower() else " не равна ")) # Исходные строки


Строка "привет мир"   равна  "привет мир"
Строка "ПРИВЕТ МИР"  равна  "ПРИВЕТ МИР"
Строка "Привет Мир"  равна  "Привет МИР"


А как оценить процентное соотношение равенства двух строк?
Функция thefuzz.fuzz.ratio()

In [5]:
a = thefuzz.fuzz.ratio(str_A , str_B)
print(a,"%") # переменная int + строка

86 %


Формула определения Равенства строк fuzz.ratio(str_A,str_B)
        2*кол-во совпадений/сумму длин двух строк

In [6]:
def distance(a, b):
    """Вычисление расстояния между строками алгоритм Левенштейна
    Редакционное расстояние"""
    n, m = len(a), len(b)   #Определение длин строк
    if n > m:
        a, b = b, a
        n, m = m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]
distance_levenstain = distance(str_A, str_B)
distance_levenstain_lower = distance(str_A.lower(), str_B.lower())
print("Расстояние между строками -",distance_levenstain)
print("Расстояние между строками -",distance_levenstain_lower)

Расстояние между строками - 2
Расстояние между строками - 1


In [7]:
def leven_ratio(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.ratio()"
    return str((2*(max(len(a),len(b))-distance(a, b))/(len(a + b)))*100) # формула которая напмсана выше в markdown
print("Расстояние между строк - " +str(distance(str_A , str_B)))
print(leven_ratio(str_A,str_B),"%")
print(leven_ratio(str_A.lower(), str_B.lower()),"%")

Расстояние между строк - 2
85.71428571428571 %
95.23809523809523 %


Функция thefuzz.fuzz.partial_ratio()

In [8]:
str_A = 'Привет мир'
str_B = 'Приве1'
thefuzz.fuzz.ratio(str_A , str_B)
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")

Обычное сравнение 62 %
Частичное сходство строк 83 %


In [9]:
def part_ratio(a,b):   # может это дать на самостоятельную, кастрированная версия поиска подстроки в строке?? либо дать ее доделать чтобы в итоге получить процент при том что смотреть пример выше!
    "Очень простой аналог функции thefuzz.fuzz.partial_ratio"
    if len(a) >= len(b): #Определение какая из строк длиннее
        short, long = b,a
    else:
        short, long = a,b
    for i in range(len(long)-len(short)+1) :
        if short == long[i:(i+len(short))]:
            return True # Тут нужно дать самостоятельную чтобы считались %
        else :
            continue
str_A = 'Привет мир'
str_B = 'мир'
print(part_ratio(str_A , str_B))

True


Функция thefuzz.fuzz.token_sort_ratio()

In [10]:
str_A = 'Привет     мир наш'
str_B = 'наш мир привет'
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")
print("Сравнение в разном порядке",thefuzz.fuzz.token_sort_ratio(str_A , str_B),"%")

Обычное сравнение 31 %
Частичное сходство строк 36 %
Сравнение в разном порядке 100 %


In [11]:
def token_sort(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.token_sort_ratio()"
    sort_a = sorted(a.lower().split())  #.split() разбиение строки в список с разделителем пробела
    sort_b =  sorted(b.lower().split()) # sorted() сортировка элементов разбитого списка в алфавитном порядке
    a = " ".join(sort_a) # конкатенация элементов списка через " " пробел
    b = " ".join(sort_b)
    if len(a) == len(b):
        return leven_ratio(a,b)
    else:
        return ("в разработке") # тут должна быть доделанная версия def part_ratio(a,b):
print(token_sort(str_A,str_B))

100.0


Функция thefuzz.fuzz.token_set_ratio()

In [12]:
str_A = 'mariners vs angels'
str_B = 'los angeles angels of anaheim at seattle mariners'
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")
print("Сравнение в разном порядке",thefuzz.fuzz.token_sort_ratio(str_A , str_B),"%")
print("Сравнение наборов токенов",thefuzz.fuzz.token_set_ratio(str_A , str_B),"%")

Обычное сравнение 36 %
Частичное сходство строк 61 %
Сравнение в разном порядке 51 %
Сравнение наборов токенов 91 %


In [13]:
def token_set_ratio(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.token_set_ratio()"
    plenty_sort_a = set(str_A.lower().split())
    plenty_sort_b = set(str_B.lower().split())
    intersection_plenty= " ".join(sorted(plenty_sort_a.intersection(plenty_sort_b)))
    # a.intersection(b) Определение пересечения двух списков (множеств)
    difference_plently = " ".join(sorted(plenty_sort_a.difference(plenty_sort_b)))
    difference_plently2 = " ".join(sorted(plenty_sort_b.difference(plenty_sort_a)))
    # a.difference(b) Определение несовпадающих элементов списка а в списке b  (множеств)
    combination1 = intersection_plenty + ' ' + difference_plently
    combination2 = intersection_plenty + ' ' + difference_plently2
    pairwise = [
        leven_ratio(intersection_plenty, combination1),
        leven_ratio(intersection_plenty, combination2),
        leven_ratio(combination1, combination2)
       ]
    return max(pairwise)

token_set_ratio(str_A,str_B)

'90.9090909090909'

In [14]:
str_A = 'Привет наш мир чй'
str_B = 'ПриВЕт наш мир!'
print(len(str_A),len(str_B))
a = thefuzz.fuzz.WRatio(str_A, str_B)
# не разобрался как она получает процент (как его посчитать руками) и как ее можно реализовать алгоритмом
print(a," %")

17 15
95  %


Функции process.extract() и extractOne() используются для исправления орфографических ошибок в MS Word либо в мобильном телефоне для автозамены или подсказок продолжения написания слова

In [15]:
str_1 = 'Пр'
list_of_strings = [' !Привет', " !Приветствую", " Приветули"]
process.extract(str_1, list_of_strings,scorer=thefuzz.fuzz.ratio , limit = 2) # параметр лимит указывает на количество возвращаемых элементов имеющих больший %, если его не указывать явно дефолту =5

[(' !Привет', 50), (' Приветули', 36)]

In [16]:
def re_extract(a, b):
    'Аналог функции process.extract, автоматически меняет переменные местами в случае неправильного порядка передачи параметров'
    if type(a) == list:
        a,b = b,a
    answer = []
    a_sort = re.findall(r'[а-яА-Я0-9]+', a.lower()) # возвращает список строк в которых найдены буквы а-я или заглавные А-Я а также цифры от 0-9, + после [] значит что конкатенировать найденные символы, без него это будет список из элементов по 1 символу
    for i in b:
        b_sort = re.findall(r'[а-яА-Я0-9]+', i.lower())
        answer.append((leven_ratio(a_sort[0],b_sort[0]),i))
    print(sorted(answer ,reverse=True)) # reverse=True значит сортировка по убыванию, по дефлту он False и по возрастанию
re_extract(str_1, list_of_strings) # сначала строка потом список
re_extract(list_of_strings, str_1) # сначала список потом строка

[('50.0', ' !Привет'), ('36.36363636363637', ' Приветули'), ('30.76923076923077', ' !Приветствую')]
[('50.0', ' !Привет'), ('36.36363636363637', ' Приветули'), ('30.76923076923077', ' !Приветствую')]


In [18]:
str_1 = 'Пр'
list_of_strings = [' !Привет', " !Приветствую", " Приветули"]
process.extractOne(str_1, list_of_strings,scorer=thefuzz.fuzz.ratio) # возвращает только самый большой по % элемент списка поэтому extractOne

(' !Привет', 50)

'тв'